In [1]:
import utils.Encryption as cen
import utils.CustomModel as cm
import utils.DataPipeline as dp
import tensorflow as tf
from PIL import Image
import json
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

d:\ProgramData\Anaconda3\envs\isic\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
root_dir = "F:\\pps_paper\\dataset"
train_dir = f"{root_dir}\\train"
test_dir = f"{root_dir}\\test"
validation_dir = f"{root_dir}\\validation"

train_images = f"{root_dir}\\train\\DataE"
train_masks = f"{root_dir}\\train\\GroundTruth_1E"
test_images = f"{root_dir}\\test\\DataE"
test_masks = f"{root_dir}\\test\\GroundTruth_1E"
validation_images = f"{root_dir}\\validation\\DataE"
validation_masks = f"{root_dir}\\validation\\GroundTruth_1E"

In [3]:
train = dp.CustomDataset(train_dir).get_Dataset()
test = dp.CustomDataset(test_dir).get_Dataset()
validation = dp.CustomDataset(validation_dir).get_Dataset()

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [16]:
model = cm.CustomModel((256,256,3,))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=optimizer)

In [17]:
history = model.fit(train, validation_data=validation,batch_size=10,epochs=3)

Epoch 1/3
2000/2000 [==============================] - 36s 17ms/step - loss: 0.0012 - val_loss: 2.6631e-06
Epoch 2/3
2000/2000 [==============================] - 34s 17ms/step - loss: 2.9675e-06 - val_loss: 2.6631e-06
Epoch 3/3
2000/2000 [==============================] - 36s 18ms/step - loss: 2.9675e-06 - val_loss: 2.6631e-06


In [18]:
history.model.summary()

Model: "custom_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(10, 256, 256, 3)]       0         
_________________________________________________________________
conv2d_24 (Conv2D)           multiple                  608       
_________________________________________________________________
conv2d_25 (Conv2D)           multiple                  1168      
_________________________________________________________________
max_pooling2d_12 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_26 (Conv2D)           multiple                  8224      
_________________________________________________________________
max_pooling2d_13 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_27 (Conv2D)           multiple               

In [8]:
img = cv.imread('F:\\pps_paper\\dataset\\train\\DataE\\ISIC_0000000.jpg',1)

key = None
with open('key.json','r') as fp:
    key = json.load(fp)

Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 20
ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

res2 = cen.unscramble(res2,key)

images = np.concatenate((img, res2), axis=1)

cv.imshow('Images', images)
 
cv.waitKey(0)
cv.destroyAllWindows()